# Max-Cut with Cirq
This notebook contains two problems of finding the max-cut in a graph.
Developed these codes thanks to `Womanium` summer 2023.

In [1]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Graph 0
Implement the Grover algorithm for the following graph:

<img src="../assets/completebipartite.png" width="25%" align="center">


In [2]:
import cirq
from cirq import H, Z, X, inverse, I, CX, CCX
s = cirq.Simulator()

# 0-3: vertices
# 4-7: edge checking
# 8-10: the number
# 11: auxillary

def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])
    
def oracle_computation():
    # EDGE CHECKS
    # Edge 1: 0-2
    yield edge_check(0, 2, 4)
    # Edge 2: 0-3
    yield edge_check(0, 3, 5)
    # Edge 3: 2-1
    yield edge_check(1, 2, 6)
    # Edge 4: 1-3
    yield edge_check(1, 3, 7)

    # SUMS
    # add edge 0
    yield CX(qq[4], qq[8])

    # add edge 1
    yield CCX(qq[5], qq[8], qq[9])
    yield CX(qq[5], qq[8])

    # add edge 2
    yield CCX(qq[6], qq[8], qq[9])
    yield CX(qq[6], qq[8])

    # add edge 3
    yield X(qq[10]).controlled_by(qq[7], qq[8],qq[9])
    yield CCX(qq[7], qq[8], qq[9])
    yield CX(qq[7], qq[8])
    
    # Only mark 4 edges = 001
    yield X(qq[8])
    yield X(qq[9])
    yield X(qq[11]).controlled_by(*qq[8:11])  

    
def oracle(): 
    yield oracle_computation()
    yield Z(qq[11])
    yield inverse(oracle_computation())
    
def grover_diffusion():
    yield H.on_each(*(qq[:4]))
    yield X.on_each(*(qq[:4]))
    yield Z(qq[3]).controlled_by(qq[0], qq[1], qq[2])
    yield X.on_each(*(qq[:4]))
    yield H.on_each(*(qq[:4]))

# the Grover algorithm
qq = cirq.LineQubit.range(12) 
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:4])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
circuit.append(cirq.measure(*qq[0:4], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**4)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

Random guess probability: 0.0625
Probability of observing 1100 :  0.4699
Probability of observing 0011 :  0.4741
Probability of observing 1110 :  0.0044
Probability of observing 0000 :  0.0046
Probability of observing 1010 :  0.0038
Probability of observing 1101 :  0.0039
Probability of observing 1011 :  0.004
Probability of observing 1001 :  0.0043
Probability of observing 0001 :  0.0041
Probability of observing 0010 :  0.0037
Probability of observing 0101 :  0.0043
Probability of observing 0111 :  0.0029
Probability of observing 1111 :  0.0042
Probability of observing 0110 :  0.0045
Probability of observing 1000 :  0.0034
Probability of observing 0100 :  0.0039


## Graph 1
$G$ has 5 vertices. Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla.

The list of edges:
(0,3)
(0,4)
(1,3)
(1,4)
(2,3)
(2,4)

In [3]:
# Diffusion operator
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [4]:
# Using the usefull edge check from the notebooks :P
def edge_check(a, b, c, qq):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])
    
def oracle2(qq):
    # EDGE CHECKS
    yield edge_check(0, 3, 5, qq)
    yield edge_check(0, 4, 6, qq)
    yield edge_check(1, 3, 7, qq)
    yield edge_check(1, 4, 8, qq)
    yield edge_check(2, 3, 9, qq)
    yield edge_check(2, 4, 10, qq)
    
    # Mark element
    yield X(qq[11]).controlled_by(*qq[5:11])
        

In [5]:
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [6]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [7]:
shots=1000
grover2(shots)

Counter({'00011': 462,
         '11100': 440,
         '01111': 2,
         '10001': 6,
         '10010': 5,
         '10110': 8,
         '01001': 4,
         '00010': 1,
         '00100': 3,
         '00110': 4,
         '11011': 6,
         '01100': 3,
         '11000': 4,
         '00111': 3,
         '10101': 6,
         '00000': 4,
         '01011': 4,
         '00001': 3,
         '01010': 6,
         '11001': 5,
         '10011': 4,
         '01101': 1,
         '01110': 1,
         '01000': 3,
         '10000': 3,
         '00101': 3,
         '11010': 3,
         '11101': 1,
         '11110': 1,
         '10111': 1})

Max-cut is 11100 or conversely 00011.

## Graph 2
$G$ has 4 vertices. Use qubits 0-3 for the vertices, 4-8 for the edges, 9-11 for the addition and 12 as the ancilla.

The list of edges: 
(0,1)
(0,2)
(0,3)
(1,2)
(1,3)

In [8]:
# Using the usefull edge check from the notebooks :P
def edge_check(a, b, c, qq):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])
    
def oracle3(qq):
    # EDGE CHECKS
    yield edge_check(0, 1, 4, qq)
    yield edge_check(0, 2, 5, qq)
    yield edge_check(0, 3, 6, qq)
    yield edge_check(1, 2, 7, qq)
    yield edge_check(1, 3, 8, qq)
    
    # SUMS
    # Add edge 4    
    yield CX(qq[4], qq[9])
    
    # Add edge 5-6
    for i in range(5, 7):
        yield X(qq[10]).controlled_by(qq[i], qq[9])
        yield CX(qq[i], qq[9])
    
    # Add edge 7-8
    for i in range(7, 9):
        yield X(qq[11]).controlled_by(qq[i], qq[9], qq[10])
        yield X(qq[10]).controlled_by(qq[i], qq[9])
        yield CX(qq[i], qq[9])
    
    # Only mark 4 edges = 001
    yield X(qq[9])
    yield X(qq[10])
    yield X(qq[12]).controlled_by(*qq[9:12]) 
    

In [9]:
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))  

In [10]:
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [11]:
shots=1000
grover3(shots)

Counter({'1010': 2,
         '0011': 462,
         '1100': 487,
         '0010': 7,
         '1001': 7,
         '1110': 8,
         '1011': 4,
         '1000': 5,
         '0001': 3,
         '0100': 2,
         '0000': 1,
         '0110': 2,
         '0101': 5,
         '0111': 1,
         '1111': 2,
         '1101': 2})

Max cut is 1100 or 0011